# 모듈 임포트


In [ ]:
# !pip install -U pandas-profiling
# !sudo apt-get install -y fonts-nanum
# !sudo fc-cache -fv
# !rm ~/.cache/matplotlib -rf

In [ ]:
import numpy as np
import pandas as pd
import pandas_profiling
import re
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib as mpl
from matplotlib import rcParams
from IPython.display import set_matplotlib_formats
set_matplotlib_formats('retina')
plt.rc('font', family='NanumBarunGothic') 
mpl.rc('axes', unicode_minus=False)

# 파일 불러오기

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
dataanalyst = pd.read_csv('/content/drive/My Drive/DataAnalyst.csv')
dataanalyst.drop(columns="Unnamed: 0", axis=1,inplace=True)
datascientist = pd.read_csv('/content/drive/My Drive/DataScientist.csv')
datascientist.drop(columns=["Unnamed: 0", "index"], axis=1,inplace=True)
businessanalyst = pd.read_csv('/content/drive/My Drive/BusinessAnalyst.csv', encoding='cp949')
businessanalyst.drop(columns=["Unnamed: 0", "index"], axis=1,inplace=True)
dataengineer = pd.read_csv('/content/drive/My Drive/DataEngineer.csv')

dataanalyst.shape, datascientist.shape, businessanalyst.shape, dataengineer.shape

In [ ]:
dataanalyst["division"] = "data analyst"
datascientist["division"] = "data scientist"
businessanalyst["division"] = "business analyst"
dataengineer["division"] = "data engineer"

job_raw = pd.concat([dataanalyst, datascientist, businessanalyst, dataengineer])
job_raw = job_raw.reset_index(drop="first")
print(job_raw.shape)
job_raw.head()

### ※ 데이터 불러오기 및 데이터 정보

**데이터 : 미국 데이터 과학자, 분석가, 엔지니어, 비지니스 분석가 구인광고**

**데이터 갯수 (12,172ea)**
*   Data Analyst : 2,253ea
*   Data Scientist : 3,909ea
*   Data Engineer : 2,528ea
*   Business Analyst : 4,902ea

**특성정보 (15 features)**


*   Job Title         : 직업 타이틀 
*   Salary Estimate   : 연봉
*   Job Description   : 직업 설명
*   Rating            : 평가 등급
*   Company Name      : 회사명
*   Location          : 직장 위치
*   Headquarters      : 본사 위치
*   Size              : 직원 규모
*   Founded           : 설립시기
*   Type of ownership : 회사유형(사기업, 공기업, 공공기관 등)
*   Industry          : 업종
*   Sector            : 사업 부문
*   Revenue           : 매출 정보
*   Competitors       : 경쟁업체
*   Easy Apply        : 지원 난이도 (True, -1)

# 데이터 간략히 보기

In [ ]:
job_raw.profile_report()

In [ ]:
job_raw.isna().sum() # 결측치 대신 -1로 들어가 있는 값들이 많음 -> 추후 처리 예정

In [ ]:
job_raw.info() # object type이 대다수, null 미존재(-1로 들어가있음)

In [ ]:
job_raw.describe(include="all")

In [ ]:
job_raw.head(1)

## ※ 1차 데이터 최적화 (데이터 전처리)
### "-1" 데이터 처리 

*   데이터 중 -1 로 표기된 부분은 전체 삭제처리(전체의 27.3%)
*   Competitor, Easy Apply는 추후 수정예정
*   전체 데이터가 충분하다고 판단하여 삭제 

In [ ]:
np.sort(job_raw["Rating"].unique()) # -1 이상치 혹은 결측치로 예상

In [ ]:
sns.displot(job_raw["Rating"]);

In [ ]:
# -1 이 포함된 관측치 확인 후 처리
for i in job_raw.columns:
  print("{} :".format(i), (job_raw[i] == "-1").sum())

In [ ]:
for i in job_raw.columns:
  print("{} :".format(i), (job_raw[i] == -1).sum())

* Salary Estimate   : 1     # 삭제
* Headquarters      : 30    # 삭제
* Rating            : 1271  # 삭제
* Founded           : 3274  # 삭제
* Type of ownership : 721   # 삭제
* Industry          : 1801  # 삭제
* Sector            : 1789  # 삭제
* Revenue           : 721   # 삭제
* Competitors       : 9228  # -1 : 경쟁없체가 없는 것로 판단하여 0, 그외 경쟁업체 수로 변경 예정
* Easy Apply        : 12258 # -1 : 지원하기 어려운것으로 판단하여 0, True는 1로 변경예정

In [ ]:
### feature engineering - "-1" 포함된 관측치 삭제
job = job_raw[job_raw["Rating"] != -1]
job = job[job["Industry"] != "-1"]
job = job[job["Salary Estimate"] != "-1"]
job = job[job["Headquarters"] != "-1"]
job = job[job["Founded"] != -1]
job = job[job["Size"] != "Unknown"]
job = job.reset_index(drop="first")

In [ ]:
print("삭제한 관측치 수 : ", job_raw.shape[0] - job.shape[0])
print("삭제한 관측치 % : ", round((job_raw.shape[0]-job.shape[0]) / job_raw.shape[0] * 100, 1))

In [ ]:
print("raw data 관측치 수 : ", job_raw.shape[0])
print("1차 전처리 후 관측치 수 : ", job.shape[0])

In [ ]:
job

#### Revenue(매출액), Size(직원규모) 금액 대신 등급으로 표기

*   매출액과 연봉의 경우 구간으로 나타나 있기 때문에 평균 혹은 실제 금액을 추정하기 보다 구간으로 설정하여 가시성을 높임
*   등급은 낮은 금액일 경우 1에서 클수록 증가 (매출액은 0-12, 직원 규모는 1-7)
*   매출액의 경우 Unknown / Non-Applicable 데이터는 일괄 0으로 변경 

In [ ]:
print(job["Revenue"].value_counts())
print()
print(job["Size"].value_counts())

In [ ]:
## Revenue
# Unknown / Non-Applicable -> 0
# Less than $1 million (USD) -> 1 (1,000,000)
# $1 to $5 million (USD) -> 2 (2,500,000)
# $5 to $10 million (USD) -> 3 (7,500,000)
# $10 to $25 million (USD) -> 4 (12,500,000)
# $25 to $50 million (USD) -> 5 (37,500,000)
# $50 to $100 million (USD) -> 6 (75,000,000)
# $100 to $500 million (USD) -> 7 (250,000,000)
# $500 million to $1 billion (USD) -> 8 (750,000,000)
# $1 to $2 billion (USD) -> 9 (1,500,000,000)
# $2 to $5 billion (USD) -> 10 (3,500,000,000)
# $5 to $10 billion (USD) -> 11 (7,500,000,000)
# $10+ billion (USD) -> 12 (10,000,000,000)

# # Size
# 1 to 50 employees -> 1
# 51 to 200 employees -> 2
# 201 to 500 employees -> 3
# 501 to 1000 employees -> 4
# 1001 to 5000 employees -> 5
# 5001 to 10000 employees -> 6
# 10000+ employees -> 7

In [ ]:
job["Revenue"] = job["Revenue"].replace("Unknown / Non-Applicable" , 0).replace("Less than $1 million (USD)", 1).replace("$1 to $5 million (USD)", 2).replace("$5 to $10 million (USD)", 3).replace("$10 to $25 million (USD)", 4).replace("$25 to $50 million (USD)", 5).replace("$50 to $100 million (USD)", 6).replace("$100 to $500 million (USD)", 7).replace("$500 million to $1 billion (USD)", 8).replace("$1 to $2 billion (USD)", 9).replace("$2 to $5 billion (USD)", 10).replace("$5 to $10 billion (USD)", 11).replace("$10+ billion (USD)", 12)
job["Size"] = job["Size"].replace("1 to 50 employees", 1).replace("51 to 200 employees", 2).replace("201 to 500 employees", 3).replace("501 to 1000 employees", 4).replace("1001 to 5000 employees", 5).replace("5001 to 10000 employees", 6).replace("10000+ employees", 7)

In [ ]:
job.head()

## ※ 2차 데이터 최적화 (데이터 전처리)

*   경쟁사(Competitor) : "-1"로 되어 있는 데이터는 경쟁사가 없는것으로 판단하여 0으로 처리, 그외 나머지는 경쟁사 수로 데이터 변경
*   지원난이도(Easy Apply) : "-1"은 어렵다로 판단하여 False로 변경 후 0,1로 변경
*   신규 특성 생성 : 본사, 근무하는 회사 일치할 경우 1, 다를 경우 0
*   연봉(Salary) : 평균 연봉으로 변경
*   회사이름(Company Name) : 회사명 뒤 평점 삭제
*   직업 타이틀(Job Title) : 타이틀 뒤 상세사항 삭제


In [ ]:
def engineering(df):
  ### Competitors_counts 생성
  df["Competitors"] = df["Competitors"].str.replace("-1", "")
  divided_competitors = df["Competitors"].str.split(",", expand=True)
  divided_competitors_bool = divided_competitors.astype(bool)
  divided_competitors_bool = divided_competitors_bool.astype(int)
  divided_competitors_bool["total"] = divided_competitors_bool[0] + divided_competitors_bool[1] + divided_competitors_bool[2] + divided_competitors_bool[3]
  df["Competitors_counts"] = divided_competitors_bool["total"]
  df.drop(columns="Competitors", axis=1, inplace=True)

  ### changing type(binary)
  # Easy Apply True 1, 아니면 0
  for i in range(len(df["Easy Apply"])):
    if df["Easy Apply"][i] == "-1":
      df["Easy Apply"][i] = 0
    else:
      df["Easy Apply"][i] = 1
  
  ###  Location=Headquaters
  # 본사근무 feature 생성
  df["Location=Headquarters"] = df["Location"] == df["Headquarters"]
  df["Location=Headquarters"] = df["Location=Headquarters"].astype(int)
  
  ### Salary
  # salary단위 연봉정보지만, 일부 시급으로 되어있는 부분 제거
  per_hour_index = np.array(df[df["Salary Estimate"].str.contains("Per")].index) 
  df.drop(index=per_hour_index, inplace=True)
  # Glassdoor est.정보가 아닌 employer_est 삭제
  employer_est_index = np.array(df[df["Salary Estimate"].str.contains("Employer")].index)
  df.drop(index=employer_est_index, inplace=True)
  # Salary estimate : min, max, average feature 생성
  divided_salary = df["Salary Estimate"].str.replace(r"\$|K|\.|\(|\)", "").str.replace("Glassdoor est", "").str.split("-", expand=True)
  Min_Salary = pd.to_numeric(divided_salary[0]) * 1000
  Max_Salary = pd.to_numeric(divided_salary[1]) * 1000
  df["Average_Salary"] = (Min_Salary + Max_Salary) / 2
  df["Average_Salary"] = df["Average_Salary"].astype(int)
  df.drop(columns="Salary Estimate", axis=1, inplace=True) # leakage

  
  ### Company name
  df["Company Name"] = df["Company Name"].str.split("\n", expand=True).iloc[:, 0] 

  ### Job Title
  divided_title = df["Job Title"].str.split(",", expand=True)
  divided_title = divided_title[0].str.split("-", expand=True)
  divided_title = divided_title[0].str.split("(", expand=True)
  divided_title[0] = divided_title[0].str.replace("Sr.", "Senior ").str.replace("Jr.", "Junior ")
  df["Job Title"] = divided_title[0]

  df.index = range(len(df))

  return df

job_clean = engineering(job)

In [ ]:
job_clean

In [ ]:
job_raw.shape[0] - job_clean.shape[0] # 3623 row 삭제

In [ ]:
from google.colab import files
job_clean.to_csv("data_job_group.csv")
files.download("data_job_group.csv")